### network analysis
- 논문 간의 키워드 연관성을 추출해본다.
- 키워드로 논문간의 번들을 형성할 수 있다. 
    - 논문의 트렌드를 형성할 수 있다.

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import re

In [2]:
df = pd.read_csv('references.csv')

In [3]:
df.tail()

,infor,refer
1457,KOSPI200 옵션전략지수의 성과\n권순신 | 강병진 | 정재만 | 한국파생상품학...,0
1458,KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로\n...,0
1459,Analytical Approximations of American Call Opt...,0
1460,현금흐름 위험 기반 KOSPI 수익률 횡단면 연구\n문성제 | 송준혁 | 한국파생상...,0
1461,모호성 회피성향이 예방적 노력에 미치는 영향\n홍지민 | 한국파생상품학회 | 선물연...,0


### 참고문헌이 나와있지 않은 논문의 수

In [4]:
len([i for i in df['refer'].values if i == '0'])

242

In [5]:
str(((len([i for i in df['refer'].values if i == '0']) / len(df)) * 100)) + '%'

'16.55266757865937%'

# Tuning_process

#### 우선 information 컬럼을 늘려준다.
- refer 컬럼을 split 하는 과정을 시행한다.

In [6]:
print(len(re.findall('[0-9]{1,}. \[',df['refer'].values[100])))
print(len(re.split('[0-9]{1,}. \[',df['refer'].values[100])))

31
32


In [7]:
head_ls = re.findall('[0-9]{1,}. \[',df['refer'].values[100])

In [8]:
testing_ls = re.split('[0-9]{1,}. \[',df['refer'].values[100])
process_ls = []
for i in range(len(testing_ls)):
    if re.search('\/ [0-9]{,4} \/',testing_ls[i]) : 
        process_ls.append(testing_ls[i])
process_ls[-2:]

['학술지(정기간행물)] - / 1986 / Large Shareholders and Corporate Control (94) : 461 ~ 488,',
 '학술지(정기간행물)] - / 1995 / Do Corporations Award CEO Stock Options Effectively? 39 : 237 ~ 269']

In [9]:
title_ls = []
for i in range(len(process_ls)):
    title_ls.append(head_ls[i] + process_ls[i])
title_ls[-2:]

['30. [학술지(정기간행물)] - / 1986 / Large Shareholders and Corporate Control (94) : 461 ~ 488,',
 '31. [학술지(정기간행물)] - / 1995 / Do Corporations Award CEO Stock Options Effectively? 39 : 237 ~ 269']

### information 컬럼을 늘려주는 function(generalization) 프로세스

In [10]:
#length column 을 만들어주기 위한 프로세스
final_process_ls = []

for i in range(len(df['refer'])):
    try :
        title_ls = []
        if df['refer'].values[i] != '0' : 
            head_ls = re.findall('[0-9]{1,}. \[',df['refer'].values[i])
            testing_ls = re.split('[0-9]{1,}. \[',df['refer'].values[i])
            process_ls = []
            for j in range(len(testing_ls)):
                if re.search('\/ [0-9]{,4} \/',testing_ls[j]) : 
                    process_ls.append(testing_ls[j])
            for k in range(len(process_ls)):
                title_ls.append(head_ls[k] + process_ls[k])
        else : title_ls.append('0')
        final_process_ls.append(title_ls)
    except Exception as e:
        print(i)
        print(len(process_ls),len(head_ls))
        print(e)
        break

In [11]:
df['length'] = [len(i) for i in final_process_ls]

In [12]:
new_df = pd.DataFrame()
new_df['refer'] = [j for i in final_process_ls for j in i]

In [13]:
extending_title_ls = []
for i in range(len(df)):
    for _ in range(df['length'].values[i]):
        extending_title_ls.append(df['infor'].values[i])
print(len(extending_title_ls))
new_df['infor'] = extending_title_ls

37247


In [14]:
new_df = pd.DataFrame()
new_df['infor'] = extending_title_ls
new_df['refer'] = [j for i in final_process_ls for j in i]

In [15]:
set(new_df['infor'].values) == set(df['infor'].values)
#OK
new_df

,infor,refer
0,기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로\n김병호 ...,0
1,원/달러 현물과 선물간의 일중 사전적 차익거래 수익성\n이재하 | 임상규 | 한국증...,0
2,주식 수익률의 카오스적 특성에 관한 실증연구\n장경천 | 김현석 | 한국증권학회 |...,0
3,한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석\n이정도 | 안영규 | 한국...,0
4,소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구\n박기성 | ...,0
5,지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구\n이원흠 | 최수미 |...,0
6,나스닥증권시장이 한국증시의 가격변동성에 미치는 영향\n김태혁 | 강석규 | 한국증권...,0
7,내부자거래 규제의 경제적 타당성 평가\n최도성 | 고봉찬 | 한국증권학회 | Asi...,0
8,기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-\n위정범 | 한경동 |...,0
9,자산가격변동에 대한 중앙은행의 방응함수 연구\n이민원 | 한국증권학회 | Asia-...,0


In [16]:
new_df.to_csv('extended_references.csv',index=False)

# Text Tuning process

# 우선 references 가 아닌 논문의 이름부터 튜닝한다.

infor 컬럼은 | 단위로 잘라줄 수 있고 \n 단위로 잘라줄 수 있다.

In [17]:
for idx in range(len(new_df['infor'].values)):
    if '\n' not in new_df['infor'].values[idx].split('|')[0]:
        print(idx)

In [18]:
new_title_ls = []
for idx in range(len(new_df['infor'].values)):
    testing_ls = new_df['infor'].values[idx].split('|')
    new_title_ls.append([j for i in [i.split('\n') for i in testing_ls] for j in i])
len(new_title_ls) == len(new_df)

True

### 튜닝한 데이터에 대해서 해당 프로세스의 새로운 데이터들을 넣어줄 새로운 데이터 프레임 tuned_df 를 만들어준다.

In [19]:
tuned_df = pd.DataFrame()
tuned_df['title_bundle'] = new_title_ls
tuned_df.tail()

,title_bundle
37242,"[KOSPI200 옵션전략지수의 성과, 권순신 , 강병진 , 정재만 , 한국파..."
37243,"[KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로,..."
37244,[Analytical Approximations of American Call Op...
37245,"[현금흐름 위험 기반 KOSPI 수익률 횡단면 연구, 문성제 , 송준혁 , 한국..."
37246,"[모호성 회피성향이 예방적 노력에 미치는 영향, 홍지민 , 한국파생상품학회 , ..."


### 리스트의 첫 번째 엘리먼트가 항상 논문의 이름인지 확인하는 프로세스가 필요해 보인다.

In [20]:
tuned_df['title_bundle'].values[0]

['기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로',
 '김병호 ',
 ' 한국증권학회 ',
 ' Asia-Pacific Journal of Financial Studies ',
 ' 30() ',
 ' pp.391~434 ',
 ' 2002.06 ',
 ' 경영학',
 '피인용횟수 : 84']

#### 이름이 넉넉잡아 5글자 이내라고 가정, 검사해본다.

In [21]:
for paper in tuned_df['title_bundle'].values:
    if len(paper[0]) <= 5 : 
        print(paper)
        
# 없으므로 naive 하지만, 각 리스트의 첫 번째 엘리먼트는 페이퍼 이름인 것을 확인할 수 있다.

### paper name 파싱! 

In [22]:
tuned_df['paper_name'] = [i[0] for i in tuned_df['title_bundle'].values]
tuned_df

,title_bundle,paper_name
0,"[기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로, 김병호...",기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로
1,"[원/달러 현물과 선물간의 일중 사전적 차익거래 수익성, 이재하 , 임상규 , ...",원/달러 현물과 선물간의 일중 사전적 차익거래 수익성
2,"[주식 수익률의 카오스적 특성에 관한 실증연구, 장경천 , 김현석 , 한국증권학...",주식 수익률의 카오스적 특성에 관한 실증연구
3,"[한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석, 이정도 , 안영규 , ...",한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석
4,"[소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구, 박기성 ,...",소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구
5,"[지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구, 이원흠 , 최수미...",지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구
6,"[나스닥증권시장이 한국증시의 가격변동성에 미치는 영향, 김태혁 , 강석규 , 한...",나스닥증권시장이 한국증시의 가격변동성에 미치는 영향
7,"[내부자거래 규제의 경제적 타당성 평가, 최도성 , 고봉찬 , 한국증권학회 , ...",내부자거래 규제의 경제적 타당성 평가
8,"[기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-, 위정범 , 한경동...",기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-
9,"[자산가격변동에 대한 중앙은행의 방응함수 연구, 이민원 , 한국증권학회 , As...",자산가격변동에 대한 중앙은행의 방응함수 연구


#### 리스트에서 두 번째부터 `한국증권학회` , `한국재무관리학회` , `한국파생상품학회` , `한국재무학회` 가 나오는 엘리먼트의 리스트 인덱스 전까지는 이름 즉, 저자이다.

In [23]:
tuned_df['title_bundle'].values[0]

['기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로',
 '김병호 ',
 ' 한국증권학회 ',
 ' Asia-Pacific Journal of Financial Studies ',
 ' 30() ',
 ' pp.391~434 ',
 ' 2002.06 ',
 ' 경영학',
 '피인용횟수 : 84']

##### 우선 한국증권학회 , 한국재무관리학회 , 한국파생상품학회 , 한국재무학회 가 나오는 인덱스를 찾는다.

In [24]:
testing_ls = []
for paper in tuned_df['title_bundle'].values : 
    for idx in range(len(paper)):
        if re.findall('한국증권학회|한국재무관리학회|한국파생상품학회|한국재무학회',paper[idx]) : 
            testing_ls.append(idx)
            break

In [25]:
institution_ls = []
for idx in range(len(tuned_df['title_bundle'].values)):
    institution_ls.append(tuned_df['title_bundle'].values[idx][testing_ls[idx]])

In [26]:
institution_ls[-2:]

[' 한국파생상품학회 ', ' 한국파생상품학회 ']

In [27]:
for i in institution_ls : 
    if not re.findall('한국증권학회|한국재무관리학회|한국파생상품학회|한국재무학회',i):
        print(i)

# 아무것도 출력되지 않으므로 institution_ls 에는 발행기관명만 있다는 것을 알 수 있다.

In [28]:
tuned_df['institution'] = institution_ls
tuned_df

,title_bundle,paper_name,institution
0,"[기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로, 김병호...",기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로,한국증권학회
1,"[원/달러 현물과 선물간의 일중 사전적 차익거래 수익성, 이재하 , 임상규 , ...",원/달러 현물과 선물간의 일중 사전적 차익거래 수익성,한국증권학회
2,"[주식 수익률의 카오스적 특성에 관한 실증연구, 장경천 , 김현석 , 한국증권학...",주식 수익률의 카오스적 특성에 관한 실증연구,한국증권학회
3,"[한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석, 이정도 , 안영규 , ...",한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석,한국증권학회
4,"[소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구, 박기성 ,...",소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구,한국증권학회
5,"[지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구, 이원흠 , 최수미...",지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구,한국증권학회
6,"[나스닥증권시장이 한국증시의 가격변동성에 미치는 영향, 김태혁 , 강석규 , 한...",나스닥증권시장이 한국증시의 가격변동성에 미치는 영향,한국증권학회
7,"[내부자거래 규제의 경제적 타당성 평가, 최도성 , 고봉찬 , 한국증권학회 , ...",내부자거래 규제의 경제적 타당성 평가,한국증권학회
8,"[기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-, 위정범 , 한경동...",기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-,한국증권학회
9,"[자산가격변동에 대한 중앙은행의 방응함수 연구, 이민원 , 한국증권학회 , As...",자산가격변동에 대한 중앙은행의 방응함수 연구,한국증권학회


### 이번에는 이름을 해본다. 이름은 두번째 요소부터 발행기관명 사이의 엘리먼트이다. 발행기관명의 인덱스는 testing_ls에 넣어두었다.

In [29]:
name_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    name_ls.append(val[1:testing_ls[idx]])
name_ls = [','.join(i) for i in name_ls]
# 리스트를 문자열 형식으로 바꿔준다.

In [30]:
tuned_df['author'] = name_ls

In [31]:
tuned_df

,title_bundle,paper_name,institution,author
0,"[기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로, 김병호...",기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로,한국증권학회,김병호
1,"[원/달러 현물과 선물간의 일중 사전적 차익거래 수익성, 이재하 , 임상규 , ...",원/달러 현물과 선물간의 일중 사전적 차익거래 수익성,한국증권학회,"이재하 , 임상규"
2,"[주식 수익률의 카오스적 특성에 관한 실증연구, 장경천 , 김현석 , 한국증권학...",주식 수익률의 카오스적 특성에 관한 실증연구,한국증권학회,"장경천 , 김현석"
3,"[한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석, 이정도 , 안영규 , ...",한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석,한국증권학회,"이정도 , 안영규"
4,"[소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구, 박기성 ,...",소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구,한국증권학회,박기성
5,"[지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구, 이원흠 , 최수미...",지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구,한국증권학회,"이원흠 , 최수미"
6,"[나스닥증권시장이 한국증시의 가격변동성에 미치는 영향, 김태혁 , 강석규 , 한...",나스닥증권시장이 한국증시의 가격변동성에 미치는 영향,한국증권학회,"김태혁 , 강석규"
7,"[내부자거래 규제의 경제적 타당성 평가, 최도성 , 고봉찬 , 한국증권학회 , ...",내부자거래 규제의 경제적 타당성 평가,한국증권학회,"최도성 , 고봉찬"
8,"[기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-, 위정범 , 한경동...",기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-,한국증권학회,"위정범 , 한경동"
9,"[자산가격변동에 대한 중앙은행의 방응함수 연구, 이민원 , 한국증권학회 , As...",자산가격변동에 대한 중앙은행의 방응함수 연구,한국증권학회,이민원


### 이번에는 저널명이다 testing_ls에서 1을 더해준다.

In [32]:
journal_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    journal_ls.append(val[testing_ls[idx]+1])

In [33]:
tuned_df['journal'] = journal_ls

In [34]:
tuned_df

,title_bundle,paper_name,institution,author,journal
0,"[기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로, 김병호...",기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로,한국증권학회,김병호,Asia-Pacific Journal of Financial Studies
1,"[원/달러 현물과 선물간의 일중 사전적 차익거래 수익성, 이재하 , 임상규 , ...",원/달러 현물과 선물간의 일중 사전적 차익거래 수익성,한국증권학회,"이재하 , 임상규",Asia-Pacific Journal of Financial Studies
2,"[주식 수익률의 카오스적 특성에 관한 실증연구, 장경천 , 김현석 , 한국증권학...",주식 수익률의 카오스적 특성에 관한 실증연구,한국증권학회,"장경천 , 김현석",Asia-Pacific Journal of Financial Studies
3,"[한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석, 이정도 , 안영규 , ...",한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석,한국증권학회,"이정도 , 안영규",Asia-Pacific Journal of Financial Studies
4,"[소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구, 박기성 ,...",소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구,한국증권학회,박기성,Asia-Pacific Journal of Financial Studies
5,"[지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구, 이원흠 , 최수미...",지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구,한국증권학회,"이원흠 , 최수미",Asia-Pacific Journal of Financial Studies
6,"[나스닥증권시장이 한국증시의 가격변동성에 미치는 영향, 김태혁 , 강석규 , 한...",나스닥증권시장이 한국증시의 가격변동성에 미치는 영향,한국증권학회,"김태혁 , 강석규",Asia-Pacific Journal of Financial Studies
7,"[내부자거래 규제의 경제적 타당성 평가, 최도성 , 고봉찬 , 한국증권학회 , ...",내부자거래 규제의 경제적 타당성 평가,한국증권학회,"최도성 , 고봉찬",Asia-Pacific Journal of Financial Studies
8,"[기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-, 위정범 , 한경동...",기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-,한국증권학회,"위정범 , 한경동",Asia-Pacific Journal of Financial Studies
9,"[자산가격변동에 대한 중앙은행의 방응함수 연구, 이민원 , 한국증권학회 , As...",자산가격변동에 대한 중앙은행의 방응함수 연구,한국증권학회,이민원,Asia-Pacific Journal of Financial Studies


#### 나머지 4개의 요소를 보게 되면, 33(1)는 무엇인지 모르겠고 나머지는 각각 페이지 수와, 논문 발행 년도 , 논문의 분야 카테고리 , 마지막으로 피인용횟수가 되겠다. 각각 분배해주자.

In [35]:
tuned_df['title_bundle'].values[100]

['해외전환사채 전환권가치의 정보효과',
 '정무권 ',
 ' 한국증권학회 ',
 ' Asia-Pacific Journal of Financial Studies ',
 ' 33(1) ',
 ' pp.33~68 ',
 ' 2004.03 ',
 ' 경영학',
 '피인용횟수 : 3']

### 뭔지 모르겠지만 넣어준다.

In [36]:
dont_know_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    dont_know_ls.append(val[-5])

tuned_df['dont_know'] = dont_know_ls

In [37]:
dont_know_ls[100]

' 33(1) '

In [38]:
[i for i in dont_know_ls if not re.findall(' [0-9]{2}\(.*\)',i)]

[]

### 페이지 수

In [39]:
page_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    page_ls.append(val[-4])

tuned_df['page'] = page_ls

In [40]:
[i for i in page_ls if not re.findall(' pp.',i)]

[]

### 발행 시점

In [41]:
year_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    year_ls.append(val[-3])

tuned_df['year'] = year_ls

In [42]:
[i for i in year_ls if not re.findall(' 20',i)]

[]

### 논문 분야

In [43]:
field_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    field_ls.append(val[-2])

tuned_df['field'] = field_ls

In [44]:
[i for i in field_ls if not re.findall('경영학',i)]

[]

### 피인용 횟수

In [45]:
cited_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    cited_ls.append(val[-1])

tuned_df['cited'] = cited_ls

In [46]:
[i for i in cited_ls if not re.findall('피인용횟수',i)]

[]

# 이렇게 해서 기존 new_df 라는 데이터 프레임에서 infor 컬럼의 튜닝을 끝냈다.

In [47]:
final_tuned_df = tuned_df.iloc[:,1:]
final_tuned_df.tail()

,paper_name,institution,author,journal,dont_know,page,year,field,cited
37242,KOSPI200 옵션전략지수의 성과,한국파생상품학회,"권순신 , 강병진 , 정재만",선물연구,26(2),pp.183~216,2018.05,경영학,피인용횟수 : 0
37243,KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로,한국파생상품학회,"염명훈 , 김지훈",선물연구,26(3),pp.345~369,2018.08,경영학,피인용횟수 : 0
37244,Analytical Approximations of American Call Opt...,한국파생상품학회,배광일,선물연구,26(3),pp.283~310,2018.08,경영학,피인용횟수 : 0
37245,현금흐름 위험 기반 KOSPI 수익률 횡단면 연구,한국파생상품학회,"문성제 , 송준혁",선물연구,26(3),pp.311~343,2018.08,경영학,피인용횟수 : 0
37246,모호성 회피성향이 예방적 노력에 미치는 영향,한국파생상품학회,홍지민,선물연구,26(3),pp.371~389,2018.08,경영학,피인용횟수 : 0


In [48]:
final_tuned_df.to_csv('tuned_dataframe_infor.csv',index=False)

# 이번엔 refer 컬럼에 대한 튜닝을 해보자.

우선 / 로 split 이 가능해 보인다.

#### 참고문헌이 없어서 '0' 으로 등록되지 않은 리스트 중 첫 번째가   "숫자. [학술지 형태] - "  가 아닌 경우를 찾아본다.

In [49]:
new_refer_ls = []

for idx in range(len(new_df['refer'].values)):
    new_refer_ls.append(new_df['refer'].values[idx].split(' / '))
len(new_refer_ls) == len(new_df)

True

In [50]:
set([len(i) for idx,i in enumerate(new_refer_ls) if i[0] != '0'])

#3종류의 리스트가 있다.

{3, 4, 5}

In [52]:
for i in new_refer_ls : 
    if i[0] != '0' : 
        if not re.findall('[0-9]{1,}. \[.+\] -',i[0]) : 
            print(i)
            
# 없다. 참고문헌이 있는 리스트의 첫 번째 엘리먼트는 동일한 형태를 띄고 있음을 알 수 있다.

#### 참고문헌이 없어서 '0' 으로 등록되지 않은 리스트 중 두 번째가 년도의 형태가 아닌 경우를 찾아본다.

In [53]:
for i in new_refer_ls : 
    if i[0] != '0' : 
        if not re.findall('[0-9]{4}',i[1]) : 
            print(i)
            print(len(i))
            
# 안타깝게도 존재한다. 이런 경우 일단 귀속시킨 후에 교수님께 피드백을 요청한다. 7개만 나와서 그나마 다행인듯..ㅠㅠ

['4. [학술지(정기간행물)] -', '4', 'Shareholder Preferences and Dividend Policy (45) : 993 ~ 1018,']
3
['8. [학술지(정기간행물)] -', '4', 'Two Agency Cost Explanations of Dividends (74) : 650 ~ 659,']
3
['33. [학술지(정기간행물)] - Kyle, Albert S', '53', 'Continuous auctions and insider trading', 'Econometrica 53 : 1315 ~ 1335,']
4
['27. [보고서] - Huang, J', '203', 'How much of the corporate-treasury yield spread is due to credit risk : 1 ~ 57,']
3
['6. [학술지(정기간행물)] - Bliss, R. R.', '200', 'Testing the Stability of Implied Probability Density Functions', 'Journal of Banking and Finance 26 : 381 ~ 422,']
4
['16. [학술지(정기간행물)] - Figlewski, S.', '179', 'Subjective information and market efficiency in a betting market', 'Journal of Political Economy 87 : 75 ~ 89,']
4
['11. [학술지(정기간행물)] - Ait-Sahalia, Y.', '200', 'How Often to Sample a Continuous-Time Process in the Presence of Market Microstructure Noise', 'Review of Financial Studies 18 : 351 ~ 416,']
4


### 3개짜리의 구조
- 1. 논문의 성향
- 1.5. 논문의 저자(있는 경우도 있고 없는 경우도 있음)
- 2. 논문의 년도
- 3. 논문의 이름

In [107]:
len([i for i in new_refer_ls if len(i) == 3])

8071

In [327]:
print(len([i for i in new_refer_ls if len(i) == 3]))
[i for i in new_refer_ls if len(i) == 3][0]
# 즉, 3개짜리에는 발행기관명과 학술지명이 있는 경우, 없는 경우가 불규칙하게 분포되어 있다. 이 또한 교수님께 피드백을 받아야 하는 부분이다.

8071


['1. [학술지(정기간행물)] -',
 '2001',
 '은행중심 자금조달 시스템의 문제점: 은행의 유동성위기가 기업가치에 미치는 영향 14 (1) : 161 ~ 188,']

## 길이가 4개짜리인 데이터 프레임만들기

In [330]:
[i for i in tuned_new_refer if len(i) == 4][0]

['1. [학술지(정기간행물)]',
 '',
 '2001',
 '은행중심 자금조달 시스템의 문제점: 은행의 유동성위기가 기업가치에 미치는 영향 14 (1) : 161 ~ 188,']

## Characteristic of paper

In [332]:
char_paper_ls = [re.findall('\[.+\]',i[0])[0] for i in [i for i in tuned_new_refer if len(i) == 4]]

## author of paper

In [334]:
name_ls = [i[1] for i in tuned_new_refer if len(i) == 4]

## year of paper published

In [336]:
year_ls = [i[2] for i in tuned_new_refer if len(i) == 4]

## name of paper

In [338]:
paper_ls = [i[3] for i in tuned_new_refer if len(i) == 4]

In [339]:
len(char_paper_ls) == len(name_ls) == len(year_ls) == len(paper_ls)

True

In [340]:
len_4_df = pd.DataFrame()
len_4_df['char'] = char_paper_ls
len_4_df['name'] = name_ls
len_4_df['year'] = year_ls
len_4_df['paper'] = paper_ls

len_4_df

,char,name,year,paper
0,[학술지(정기간행물)],,2001,은행중심 자금조달 시스템의 문제점: 은행의 유동성위기가 기업가치에 미치는 영향 14...
1,[학술지(정기간행물)],,2000,"국내 은행간 합병의 효과분석 17 (1) : 283 ~ 306,"
2,[학술지(정기간행물)],,2000,"은행퇴출과 관계효과 상실에 관한 실증연구 5 (3) : 1 ~ 27,"
3,[학술지(정기간행물)],,2002,"관계금융효과의 비대칭성에 관한 실증연구: 제일은행 사례를 중심으로 : 425 ~ 463,"
4,[학술지(정기간행물)],,1999,The Consolidation of the Financial Services In...
5,[학술지(정기간행물)],,1998,The Effects of Bank Mergers and Acquisitions o...
6,[학술지(정기간행물)],,1995,The Effect of Lender Identity on a Borrowing F...
7,[학술지(정기간행물)],,2000,Relationship Banking: What Do We Know? 9 : 7 ~...
8,[학술지(정기간행물)],,1998,A Research Note on the Theory of SME: Bank Rel...
9,[학술지(정기간행물)],,2001,"Banking Market Structure, Financial Dependence..."


---------------

# 4개짜리의 구조
- 마지막 엘리먼트는 저널명이 되겠다.

In [90]:
[i for i in new_refer_ls if len(i) == 4][800]

['9. [학술지(정기간행물)] - Kreps, D',
 '1982',
 'Reputation and Imperfect Information',
 'Journal of Economic Theory 27 : 253 ~ 279,']

In [100]:
set([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]])

{'Pacific-Basin Finance Journal 10 : 411 ~ 442,',
 'Journal of Derivatives 21 (9) : 30 ~ 40,',
 '재무연구 28 (2) : 309 ~ 350,',
 'Journal of Economic Behavior and Organization 1 : 39 ~ 60,',
 'Journal of Finance 54 : 1045 ~ 1082,',
 'Journal of Finance 50 : 281 ~ 300,',
 'Accounting Review 67 : 183 ~ 198,',
 'International Review of Financial analysis 13 : 13 ~ 25',
 'Accounting Review 59 (4) : 604 ~ 618,',
 'Journal of Business 79 : 37 ~ 59,',
 'Journal of Accounting and Economics 17 : 69 ~ 94,',
 'Journal of Financial Economics 109 (1) : 103 ~ 121,',
 'Journal of Financial Economics 46 (1) : 29 ~ 65,',
 '재무관리연구 8 (2) : 73 ~ 97,',
 'Journal of Financial and Quantitative Analysis 32 : 463 ~ 489,',
 'Journal of Banking & Finance 51 : 131 ~ 150,',
 'Lab Econ 8 : 667 ~ 689,',
 'Journal of Financial Markets 14 : 127 ~ 160,',
 'Seoul Journal of Business 20 (1) : 91 ~ 138,',
 'International Review of Financial Analysis 8 : 21 ~ 34,',
 '경제연구 22 (4) : 103 ~ 145,',
 'Journal of Financial Economics 

In [127]:
print(len([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]]))

28840


- 아래는 길이가 4개인 리스트에서 저널명을 나타내는 마지막 리스트를 빼온 것이다.
- 이 리스트를 통해서 길이가 3개짜리인 리스트의 마지막에서 저널명이 포함된 것을 빼와본다.
- 우선 일반화를 위해서 숫자를 다 빼고 유니크한 값으로 변환시켜준다.

In [161]:
[paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]][-5:]

['Journal of Finance 58 (3) : 1113 ~ 1137,',
 'Journal of Political Economy 111 (6) : 1183 ~ 1220,',
 'Journal of Finance 52 (1) : 35 ~ 55,',
 'Journal of Financial and Quantitative Analysis 34 (1) : 89 ~ 114,',
 'Journal of Business 75 (4) : 583 ~ 608']

In [162]:
len([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]])

28840

좀 걸린다.

In [163]:
extract_journal_ls = []
for idx in range(28840):
    extract_journal_ls.append(\
','.join(re.findall('\D',[paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]][idx]))\
.replace(',','').split(' : ')[0])
                              
len(extract_journal_ls) == len([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]])

True

In [164]:
extract_journal_ls = [i.replace('(','') for i in extract_journal_ls]
extract_journal_ls = [i.replace(')','') for i in extract_journal_ls]
print(len(extract_journal_ls))
extract_journal_ls = list(set(extract_journal_ls))
print(len(extract_journal_ls))

28840
3048


In [169]:
extract_journal_ls

['',
 'IUP Journal of Applied Finance  ',
 'Bell Journal of Economics and Management Science  ',
 'The Journal of Derivatives Fall',
 '한국증권학회 제차 정기학술발표회 발표논문',
 '중국연구 ',
 'RAND Journal of Economics Autumn',
 'The Bell Journal of Economics ',
 'American Economic Reivew ',
 'Business Lawyer  ',
 'Central Banking Journal',
 '법무법인세종',
 'Journal of Financial Stability  ',
 'Harper Collins',
 'Journal of Financial Economics  ',
 'IIMB Management Review ',
 'The International Economy',
 'British Medical Journal ',
 'National Bureau of Economic Research',
 'Journal of the European Economic Association  ',
 '주간 금융브리프',
 '증권예탁 ',
 '인권과 정의 ',
 'Journal of Financial Services Research  -',
 '자본시장 Weekly',
 '대한경영학회지 ',
 'Journal of Political Economy ',
 'Accounting Journal ',
 'British Accounting Review ',
 '년 재무학회 정기학술대회',
 'Asia-PacificJournal of Finance Studies ',
 'Global Finance Journal  ',
 'World Development ',
 '한국증권학회 년 추계학술발표',
 'Pacific Basin Finance Journal  ',
 'Journal of Economics and

# 4개짜리 데이터 프레임으로 만들기

In [273]:
[i for i in tuned_new_refer if len(i) == 5][0]

['4. [학술지(정기간행물)]',
 ' 정완호',
 '2003',
 '기업 신용위험의 측정- 채권수익률을 이용하여 -',
 'Asia-Pacific Journal of Financial Studies 32 (1) : 135 ~ 172,']

## Characteristic of paper

In [270]:
char_paper_ls = [re.findall('\[.+\]',i[0])[0] for i in [i for i in tuned_new_refer if len(i) == 5]]

## author of paper

In [281]:
name_ls = [i[1] for i in tuned_new_refer if len(i) == 5]

## year of paper published

In [283]:
year_ls = [i[2] for i in tuned_new_refer if len(i) == 5]

## name of paper

In [285]:
paper_ls = [i[3] for i in tuned_new_refer if len(i) == 5]

## journal name which paper published

In [286]:
journal_ls = [i[-1] for i in tuned_new_refer if len(i) == 5]

In [287]:
len(char_paper_ls) == len(name_ls) == len(year_ls) == len(paper_ls) == len(journal_ls)

True

In [288]:
len_5_df = pd.DataFrame()
len_5_df['char'] = char_paper_ls
len_5_df['name'] = name_ls
len_5_df['year'] = year_ls
len_5_df['paper'] = paper_ls
len_5_df['journal'] = journal_ls

len_5_df

,char,name,year,paper,journal
0,[학술지(정기간행물)],정완호,2003,기업 신용위험의 측정- 채권수익률을 이용하여 -,Asia-Pacific Journal of Financial Studies 32 (...
1,[학술지(정기간행물)],신현한,2004,신규공모주의 저평가 발행과 시장조성제도,Asia-Pacific Journal of Financial Studies 33 (...
2,[학술지(정기간행물)],연강흠,2002,코스닥시장의 일률적 락업제도에 대한 실효성 검증,Asia-Pacific Journal of Financial Studies 31 :...
3,[학술지(정기간행물)],나영,2003,IMF이후 신용등급예측에 있어서 재무정보의 유용성,"회계정보연구 21 : 211 ~ 235,"
4,[학술지(정기간행물)],최정호,2005,회계제도개선과 감사품질이 재량적 발생의 크기와 정보성에 미치는 영향,"회계학연구 30 (2) : 107 ~ 149,"
5,[학술지(정기간행물)],"Amihud, Y",1986,Asset pricing and the bid-ask spread,"Journal of Financial Economics 17 : 223 ~ 247,"
6,[학술지(정기간행물)],"Andersen, T.G",1996,Return volatility and trading volume: an infor...,"Journal of Finance 51 : 169 ~ 204,"
7,[학술지(정기간행물)],"Bailey, W",1994,Risk and return on China’s new stock markets: ...,"Pacific-Basin Finance Journal 2 : 243 ~ 260,"
8,[학술지(정기간행물)],"Bailey, W",1999,Foreign ownership restrictions and equity pric...,Journal of Financial and Quantitative Analysis...
9,[학술지(정기간행물)],"Bailey, W",1994,Foreign ownership restrictions and stock price...,"Journal of Financial Economics 36 : 57 ~ 87,"


-------------------------------------

# 5개짜리의 구조
- 5개짜리는 이유가 보다 복잡하다.

In [58]:
len([i for i in new_refer_ls if len(i) == 5])

94

- 우선 첫 번째 리스트에서 - 이후에 성명이 나와있는 경우가 있다.
- 마지막과 그 전 리스트 엘리먼트는 해당 저자의 소속과 학위가 나와있다.

# 학위

In [96]:
set([paper[-2] for paper in [i for i in new_refer_ls if len(i) == 5]])

{'Ph.D', '博士', '박사', '석사'}

# 소속

In [97]:
set([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 5]])

{'Cornell University,',
 'Florida State Univ,',
 'KAIST',
 'KAIST,',
 'National University of Singapore,',
 'Seoul National University,',
 'The University of Chicago,',
 'The University of Texas at Austin,',
 'UZH and ETHZ,',
 'University of California,',
 'University of Chicago,',
 'University of Southern California',
 '天津大学,',
 '朝鮮大學校 大學院,',
 '武汉大学,',
 '高麗大學校 大學院,',
 '건국대학교 대학원,',
 '건국대학교,',
 '경북대학교 대학원,',
 '경희대학교 대학원,',
 '계명대학교 대학원,',
 '고려대학교 경영대학원,',
 '고려대학교 대학원,',
 '고려대학교,',
 '국민대학교,',
 '대구대학교,',
 '동국대학교 대학원,',
 '명지대대학원,',
 '부경대학교,',
 '부산대학교,',
 '서울대학교 대학원,',
 '서울대학교,',
 '서울시립대학교 대학원,',
 '서울시립대학교,',
 '성균관대학교 대학원,',
 '성균관대학교,',
 '성균관대학교대학원,',
 '숭실대학교,',
 '아주대학교 공공정책대학원,',
 '아주대학교,',
 '연세대학교 경제대학원,',
 '연세대학교 대학원,',
 '영남대학교 대학원,',
 '인하대학교,',
 '전남대학교 대학원,',
 '전남대학교,',
 '전북대학교,',
 '전주대학교 대학원,',
 '한국과학 기술원,',
 '한국과학기술원,',
 '한국외국어대학교 대학원,',
 '한국외국어대학교,',
 '한양대학교 대학원,'}

### 우선 이름이 있는 것을 뽑아주기 위해서 첫 번째 리스트를 한 번 더 세분화시켜준다.

In [65]:
tuned_new_refer = []
for idx in range(len(new_refer_ls)):
    tuned_new_refer.append(new_refer_ls[idx][0].split(" -") + new_refer_ls[idx][1:])
len(tuned_new_refer) == len(new_refer_ls)

True

In [257]:
merge = [(1, 3)]

for ls in tuned_new_refer : 
    if ls[0] != '0': 
        if not re.findall('[0-9]{1,4}',ls[2]) : 
            for t in merge[::-1]:
                merged = ''.join(ls[t[0]:t[1]])  # merging values within a range
                ls[t[0]:t[1]] = [merged]

In [258]:
for ls in tuned_new_refer : 
    if ls[0] != '0': 
        if not re.findall('[0-9]{1,4}',ls[2]) : 
            print(ls)

In [259]:
set([len(i) for idx,i in enumerate(tuned_new_refer) if i[0] != '0'])

#3종류의 리스트가 있다.

{4, 5, 6}

# 길이가 6개인 리스트를 나눠서 데이터 프레임으로 넣기

In [260]:
[i for i in tuned_new_refer if len(i) == 6][0]

['22. [학위논문]',
 ' Low, A',
 '1992',
 'Essays on asymmetric information in international finance',
 '박사',
 'University of California,']

## Characteristic of paper

In [261]:
char_paper_ls = [re.findall('\[.+\]',i[0])[0] for i in [i for i in tuned_new_refer if len(i) == 6]]

## Author of paper

In [262]:
name_ls = []

for k in range(len([i for i in tuned_new_refer if len(i) == 6])):
    name_ls.append([i for i in tuned_new_refer if len(i) == 6][k]\
[1:[idx for idx,val in enumerate([i for i in tuned_new_refer if len(i) == 6][k]) if re.findall("[0-9]{4}",val)][0]][0])

## Year of paper published

In [263]:
year_ls = []

for k in range(len([i for i in tuned_new_refer if len(i) == 6])):
    year_ls.append([i for i in tuned_new_refer if len(i) == 6][k]\
[[idx for idx,val in enumerate([i for i in tuned_new_refer if len(i) == 6][k]) if re.findall("[0-9]{4}",val)][0]])

## Name of paper

In [264]:
paper_ls = []

for k in range(len([i for i in tuned_new_refer if len(i) == 6])):
    paper_ls.append([i for i in tuned_new_refer if len(i) == 6][k]\
[[idx+1 for idx,val in enumerate([i for i in tuned_new_refer if len(i) == 6][k]) if re.findall("[0-9]{4}",val)][0]])

## Details about Author (degree , school)

In [265]:
degree_ls = [i[-2] for i in tuned_new_refer if len(i) == 6]
school_ls = [i[-1] for i in tuned_new_refer if len(i) == 6]

In [266]:
len(char_paper_ls) == len(name_ls) == len(year_ls) == len(paper_ls) == len(degree_ls) == len(school_ls)

True

In [267]:
len_6_df = pd.DataFrame()
len_6_df['char'] = char_paper_ls
len_6_df['name'] = name_ls
len_6_df['year'] = year_ls
len_6_df['paper'] = paper_ls
len_6_df['degree'] = degree_ls
len_6_df['school'] = school_ls

len_6_df

,char,name,year,paper,degree,school
0,[학위논문],"Low, A",1992,Essays on asymmetric information in internatio...,박사,"University of California,"
1,[학위논문],"Averbukh, V",1997,Pricing American options using Monte Carlo sim...,박사,"Cornell University,"
2,[학위논문],박지홍,2009,한국 펀드시장에서의 스마트머니 효과에 대한 실증연구,석사,"한국과학 기술원,"
3,[학위논문],한기문,2010,국제팩토링시스템의 이용에 관한 연구 : 중소기업의 수출금융을 중심으로,박사,"성균관대학교 대학원,"
4,[학위논문],"Chen, H",2007,"Macroeconomic Conditions, Corporate Finance De...",박사,"The University of Chicago,"
5,[학위논문],김경용,2013,한달 내의 최대 일별수익률과 주식 기대수익률의 관계,석사,"한국과학기술원,"
6,[학위논문],한형호,2003,벤처캐피탈 특성과 IPO 벤처기업의 장·단기 성과와의 관계에 관한 실증 연구,박사,"건국대학교,"
7,[학위논문],한기문,2010,국제팩토링시스템의 이용에 관한 연구 : 중소기업의 수출금융을 중심으로,박사,"성균관대학교 대학원,"
8,[학위논문],김석봉,2014,신주발행을 위한 유상증자시 발행가격 저평가 정도에 관한 연구 : 대주주와 투자은행의...,박사,"고려대학교 대학원,"
9,[학위논문],"Chen, H",2007,"Macroeconomic Conditions, Corporate Finance De...",박사,"The University of Chicago,"


# concatenate the 3 dataframe

In [342]:
final_df = pd.concat([len_4_df,len_5_df],axis=0)
final_df = pd.concat([final_df,len_6_df],axis=0)
final_df.reset_index(drop=True,inplace=True)
final_df

,char,degree,journal,name,paper,school,year
0,[학술지(정기간행물)],NaN,NaN,,은행중심 자금조달 시스템의 문제점: 은행의 유동성위기가 기업가치에 미치는 영향 14...,NaN,2001
1,[학술지(정기간행물)],NaN,NaN,,"국내 은행간 합병의 효과분석 17 (1) : 283 ~ 306,",NaN,2000
2,[학술지(정기간행물)],NaN,NaN,,"은행퇴출과 관계효과 상실에 관한 실증연구 5 (3) : 1 ~ 27,",NaN,2000
3,[학술지(정기간행물)],NaN,NaN,,"관계금융효과의 비대칭성에 관한 실증연구: 제일은행 사례를 중심으로 : 425 ~ 463,",NaN,2002
4,[학술지(정기간행물)],NaN,NaN,,The Consolidation of the Financial Services In...,NaN,1999
5,[학술지(정기간행물)],NaN,NaN,,The Effects of Bank Mergers and Acquisitions o...,NaN,1998
6,[학술지(정기간행물)],NaN,NaN,,The Effect of Lender Identity on a Borrowing F...,NaN,1995
7,[학술지(정기간행물)],NaN,NaN,,Relationship Banking: What Do We Know? 9 : 7 ~...,NaN,2000
8,[학술지(정기간행물)],NaN,NaN,,A Research Note on the Theory of SME: Bank Rel...,NaN,1998
9,[학술지(정기간행물)],NaN,NaN,,"Banking Market Structure, Financial Dependence...",NaN,2001


In [344]:
final_df.to_csv('tuned_dataframe_refer.csv',index=False)

# concatenate tuned_infor and tuned_refer dataframe

In [346]:
infor_csv = pd.read_csv("tuned_dataframe_infor.csv")
refer_csv = pd.read_csv("tuned_dataframe_refer.csv")

df = pd.concat([infor_csv,refer_csv],axis=1)

In [348]:
df.columns = ['paper_name_infor', 'institution_infor', 'author_infor', 'journal_infor', 'dont_know_infor', 'page_infor',
       'year_infor', 'field_infor', 'cited_infor', 'char_refer', 'degree_refer', 'journal_refer', 'name_refer', 'paper_refer',
       'school_refer', 'year_refer']

In [350]:
df.to_csv('tuned_dataframe.csv',index=False)

In [351]:
df

,paper_name_infor,institution_infor,author_infor,journal_infor,dont_know_infor,page_infor,year_infor,field_infor,cited_infor,char_refer,degree_refer,journal_refer,name_refer,paper_refer,school_refer,year_refer
0,기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로,한국증권학회,김병호,Asia-Pacific Journal of Financial Studies,30(),pp.391~434,2002.06,경영학,피인용횟수 : 84,[학술지(정기간행물)],NaN,NaN,NaN,은행중심 자금조달 시스템의 문제점: 은행의 유동성위기가 기업가치에 미치는 영향 14...,NaN,2001.0
1,원/달러 현물과 선물간의 일중 사전적 차익거래 수익성,한국증권학회,"이재하 , 임상규",Asia-Pacific Journal of Financial Studies,30(),pp.267~296,2002.06,경영학,피인용횟수 : 2,[학술지(정기간행물)],NaN,NaN,NaN,"국내 은행간 합병의 효과분석 17 (1) : 283 ~ 306,",NaN,2000.0
2,주식 수익률의 카오스적 특성에 관한 실증연구,한국증권학회,"장경천 , 김현석",Asia-Pacific Journal of Financial Studies,30(),pp.99~130,2002.06,경영학,피인용횟수 : 12,[학술지(정기간행물)],NaN,NaN,NaN,"은행퇴출과 관계효과 상실에 관한 실증연구 5 (3) : 1 ~ 27,",NaN,2000.0
3,한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석,한국증권학회,"이정도 , 안영규",Asia-Pacific Journal of Financial Studies,30(),pp.33~72,2002.06,경영학,피인용횟수 : 33,[학술지(정기간행물)],NaN,NaN,NaN,"관계금융효과의 비대칭성에 관한 실증연구: 제일은행 사례를 중심으로 : 425 ~ 463,",NaN,2002.0
4,소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구,한국증권학회,박기성,Asia-Pacific Journal of Financial Studies,30(),pp.297~326,2002.06,경영학,피인용횟수 : 79,[학술지(정기간행물)],NaN,NaN,NaN,The Consolidation of the Financial Services In...,NaN,1999.0
5,지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구,한국증권학회,"이원흠 , 최수미",Asia-Pacific Journal of Financial Studies,30(),pp.327~362,2002.06,경영학,피인용횟수 : 13,[학술지(정기간행물)],NaN,NaN,NaN,The Effects of Bank Mergers and Acquisitions o...,NaN,1998.0
6,나스닥증권시장이 한국증시의 가격변동성에 미치는 영향,한국증권학회,"김태혁 , 강석규",Asia-Pacific Journal of Financial Studies,30(),pp.363~390,2002.06,경영학,피인용횟수 : 25,[학술지(정기간행물)],NaN,NaN,NaN,The Effect of Lender Identity on a Borrowing F...,NaN,1995.0
7,내부자거래 규제의 경제적 타당성 평가,한국증권학회,"최도성 , 고봉찬",Asia-Pacific Journal of Financial Studies,30(),pp.1~32,2002.06,경영학,피인용횟수 : 19,[학술지(정기간행물)],NaN,NaN,NaN,Relationship Banking: What Do We Know? 9 : 7 ~...,NaN,2000.0
8,기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-,한국증권학회,"위정범 , 한경동",Asia-Pacific Journal of Financial Studies,30(),pp.131~162,2002.06,경영학,피인용횟수 : 4,[학술지(정기간행물)],NaN,NaN,NaN,A Research Note on the Theory of SME: Bank Rel...,NaN,1998.0
9,자산가격변동에 대한 중앙은행의 방응함수 연구,한국증권학회,이민원,Asia-Pacific Journal of Financial Studies,30(),pp.227~266,2002.06,경영학,피인용횟수 : 1,[학술지(정기간행물)],NaN,NaN,NaN,"Banking Market Structure, Financial Dependence...",NaN,2001.0
